In [ ]:
from revenues import *
import copy
from multiprocessing import Pool, cpu_count
import sys
import cPickle
import os


def busy_poet(params):
    _, usefulPriceBaseline = useful_work(params)
    aUseful, aStd, aFarming = var("aUseful, aStd, aFarming")

    # these symbols are declared here in order to differentiate standard
    # operation from mining, because they have different ages.
    eCostStd, eCostFarming, costStd, costFarming, RwStd = var("eCostStd, eCostFarming, costStd, costFarming, RwStd")
    subs = [
        (eCostStd, eCost.subs(u, 1).subs(age, aStd)),
        (eCostFarming, eCost.subs(u, 0).subs(age, aFarming)),
        (costStd, cost.subs(age, aStd)),
        (costFarming, cost.subs(age, aFarming)),
        (RwStd, R_w.subs(age, aStd).subs(u, 1))
    ]

    # Standard mining: (working without mining is strictly inferior)
    perCPUCostStandard = (eCostStd + overheadStd + costStd)
    perCPURev = (RwStd + R_b - perCPUCostStandard)
    cpuCountStandard = (total_cap / perCPUCostStandard).subs(params)
    revStandard = (perCPURev * cpuCountStandard)

    # Farming: (Mining in standard form doesn't make sense)
    perCPUCostFarm = (eCostFarming + overheadFarm + costFarming)
    perCPURevFarm = (R_b - perCPUCostFarm)
    cpuCountFarm = (total_cap / perCPUCostFarm).subs(params)
    revFarm = (perCPURevFarm * cpuCountFarm)

    # equilibrium:
    stdOpRatio = var("stdOpRatio")
    busyPoET_R_b = annualCryptoRev / \
        ( stdOpRatio * operatorCount * cpuCountStandard + (1 - stdOpRatio) * operatorCount * cpuCountFarm )

    busyPoET_R_b = busyPoET_R_b.subs(subs).subs(params)

    # 1) condition one: revenues are equal
    revDiff = (revStandard - revFarm).subs(R_b, busyPoET_R_b).subs(subs).subs(params)

    aSingleStd, aSingleFarmer = var("aSingleStd, aSingleFarmer")
    utilitySingleFarmer = revFarm.subs(subs).subs(aFarming, aSingleFarmer).subs(R_b, busyPoET_R_b).subs(params)
    # 2) condition two: farming age is optimal
    utilityFarmingDiff = diff(utilitySingleFarmer, aSingleFarmer).subs(aSingleFarmer, aFarming)

    utilitySingleStdMiner = revStandard.subs(subs).subs(aStd, aSingleStd).subs(R_b, busyPoET_R_b).subs(params)
    # 3) condition three: std mining age is optimal
    utilityStdMinerDiff = diff(utilitySingleStdMiner, aSingleStd).subs(aSingleStd, aStd)

    def threeConditions(arg):
        x, y, z = arg
        x = min(MAX_STALENESS, x)
        y = min(MAX_STALENESS, y)
#         if z > 1.0:
#             z = 1
#         elif z < 0.0:
#             z = 0.0

        return (revDiff.subs([(aFarming, x), (aStd, y), (stdOpRatio, z)]), 
               utilityFarmingDiff.subs(([(aFarming, x), (aStd, y), (stdOpRatio, z)])),
               utilityStdMinerDiff.subs(([(aFarming, x), (aStd, y), (stdOpRatio, z)])))

    bestAgeFarming, bestAgeStd, bestStdOpRatio = fsolve(threeConditions, [5, 5, .5])

    if bestStdOpRatio >= 1.0:
        bestStdOpRatio = 1.0
        bestAgeFarming = -1
        bestAgeStd, = fsolve(lambdify(aStd, utilityStdMinerDiff.subs(stdOpRatio, 1.0)), 5)
    elif bestStdOpRatio <= 0.0:
        bestStdOpRatio = 0.0
        bestAgeStd = -1
        bestAgeFarming, = fsolve(lambdify(aFarming, utilityFarmingDiff.subs(stdOpRatio, 0.0)), 5)
        # pprint (utilityFarmingDiff.subs(stdOpRatio, bestStdOpRatio).atoms(Symbol))
        
    bestAgeFarming = min(MAX_STALENESS, bestAgeFarming)
    bestAgeStd = min(MAX_STALENESS, bestAgeStd)
    
    cpuCountStd_v = cpuCountStandard.subs(subs).subs(params).subs(aStd, bestAgeStd)
        
    return (bestAgeFarming, bestAgeStd, bestStdOpRatio, cpuCountStd_v)
    
    
for opCount in np.logspace(0,7, num=20):
    valSubs["annualCryptoRev"] = 248.87 * 25 * 365 * 24 * 6;
    valSubs["operatorCount"] = opCount
    bestAgeFarming, bestAgeStd, bestStdOpRatio, cpuCountStd_v = busy_poet(valSubs)
    print opCount, bestStdOpRatio, opCount*bestStdOpRatio*cpuCountStd_v, bestStdOpRatio

/Users/fanz/Library/Python/2.7/lib/python/site-packages/scipy/optimize/minpack.py:236: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/Users/fanz/Library/Python/2.7/lib/python/site-packages/scipy/optimize/minpack.py:236: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)


1.0 0.0 0 0.0
